In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train_img_path = 'data/train_old'
val_img_path = 'data/test_old'
x_train = []
y_train = []
x_val = []
y_val = []

In [3]:
def load_path(img_path):
    _path = []
    lables = []
    with open(img_path, 'r') as file:
        # 将文本文件的数据读入到path_data
        path_data = file.readlines()
        for i in range(0, len(path_data)):
            # 将字符串格式转换为正常格式
            path_data[i] = path_data[i].split()
            _path.append(path_data[i][0])
            lables.append(int(path_data[i][1]))
        x = np.array(_path)
        y = np.array(lables)
        return x, y

In [4]:
x_train, y_train = load_path(train_img_path)
data_num = len(x_train)

In [5]:
def path_to_data(path, size=None):
    if size != None:
        return cv2.resize(cv2.imread(path), size)
    else:
        return cv2.resize(cv2.imread(path), (224, 224))

In [6]:
def generate_batch(x, y, batch_size=128):
    for start in range(0, data_num):
        x_batch = x[start: start + batch_size]
        y_batch = y[start: start + batch_size]
        start += batch_size

        _x_batch = []
        for i in range(x_batch.shape[0]):
            _x_batch += [path_to_data(x_batch[i]), (224, 224)]
        x_batch = np.array(_x_batch)
        yield x_batch, y_batch

In [11]:
print(generate_batch(x_train, y_train, 128))
print(x_train[0], y_train[0])

<generator object generate_batch at 0x7f588a27ad58>
data/track_data_new/no_daocha/jinan_20150921_no_daocha_left_00273.jpg 0
